# Function Calling

InteropRouter supports function calling (tool use) across providers. You define tools using OpenAI's `FunctionToolParam` type, and InteropRouter translates them to each provider's native format.

This notebook demonstrates:

- Defining custom tools
- Parallel function calling
- Built-in web search


In [1]:
import os

from anthropic import AsyncAnthropic
from google import genai
from openai import AsyncOpenAI

from interop_router.router import Router
from interop_router.types import ChatMessage, RouterResponse

router = Router()
router.register("openai", AsyncOpenAI())
router.register("gemini", genai.Client(api_key=os.getenv("GEMINI_API_KEY")))
router.register("anthropic", AsyncAnthropic())

In [2]:
import json

from openai.types.responses import EasyInputMessageParam
from openai.types.responses.response_input_item_param import FunctionCallOutput


def extract_text(response: RouterResponse) -> str:
    texts: list[str] = []
    for chat_message in response.output:
        if chat_message.message.get("type") == "message":
            content = chat_message.message.get("content")
            if isinstance(content, list):
                for c in content:
                    text = c.get("text", "")
                    if text:
                        texts.append(text)
    return "\n".join(texts)


def extract_function_calls(response: RouterResponse) -> list[dict]:
    calls = []
    for chat_message in response.output:
        if chat_message.message.get("type") == "function_call":
            calls.append(
                {
                    "call_id": chat_message.message.get("call_id", ""),
                    "name": chat_message.message.get("name", ""),
                    "arguments": chat_message.message.get("arguments", ""),
                }
            )
    return calls


def simulate_tool_execution(name: str, arguments: str) -> str:
    args = json.loads(arguments)
    if name == "get_weather":
        return json.dumps(
            {
                "temperature": 22,
                "unit": args.get("unit", "celsius"),
                "conditions": "sunny",
                "location": args.get("location", "Unknown"),
            }
        )
    elif name == "get_stock_price":
        return json.dumps(
            {
                "ticker": args.get("ticker", "UNKNOWN"),
                "price": 178.50,
                "currency": "USD",
            }
        )
    return json.dumps({"error": "Unknown tool"})

## Defining Custom Tools

Tools are defined using `FunctionToolParam` from `openai.types.responses.function_tool_param`.

See [Function calling with OpenAI](https://platform.openai.com/docs/guides/function-calling), [Function calling with the Gemini API](https://ai.google.dev/gemini-api/docs/function-calling?example=meeting), and [Tool use with Claude](https://platform.claude.com/docs/en/agents-and-tools/tool-use/overview) for provider-specific details.


In [3]:
from openai.types.responses.function_tool_param import FunctionToolParam

TOOLS: list[FunctionToolParam] = [
    FunctionToolParam(
        type="function",
        name="get_weather",
        description="Get the current weather for a given location.",
        parameters={
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "The city and country"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location", "unit"],
            "additionalProperties": False,
        },
        strict=True,
    ),
    FunctionToolParam(
        type="function",
        name="get_stock_price",
        description="Get the current stock price for a given ticker symbol.",
        parameters={
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol"},
            },
            "required": ["ticker"],
            "additionalProperties": False,
        },
        strict=True,
    ),
]

## Basic Function Calling

The function calling flow:

1. Send a request with `tools=` parameter
2. Model returns a `function_call` with `call_id`, `name`, and `arguments`
3. Execute the function and create a `FunctionCallOutput` with the result
4. Send a follow-up request to get the natural language response

We demonstrate a full roundtrip: OpenAI handles the function call, Gemini interprets the result, then Anthropic answers a follow-up.


In [4]:
messages: list[ChatMessage] = [
    ChatMessage(message=EasyInputMessageParam(role="user", content="What's the weather in Tokyo? Use celsius."))
]

# Step 1: OpenAI makes the function call
response1 = await router.create(input=messages, model="gpt-5.2", tools=TOOLS)

function_calls = extract_function_calls(response1)
print("Function calls from OpenAI:")
for call in function_calls:
    print(f"  {call['name']}({call['arguments']})")

Function calls from OpenAI:
  get_weather({"location":"Tokyo, Japan","unit":"celsius"})


In [5]:
# Step 2: Add the function call and its output to the conversation
messages.extend(response1.output)

for call in function_calls:
    result = simulate_tool_execution(call["name"], call["arguments"])
    messages.append(
        ChatMessage(
            message=FunctionCallOutput(
                call_id=call["call_id"],
                output=result,
                type="function_call_output",
            )
        )
    )
    print(f"Tool result: {result}")

Tool result: {"temperature": 22, "unit": "celsius", "conditions": "sunny", "location": "Tokyo, Japan"}


In [6]:
# Step 3: Gemini interprets the result
response2 = await router.create(input=messages, model="gemini-3-flash-preview")
messages.extend(response2.output)

print("Gemini response:")
print(extract_text(response2))

Gemini response:
The current weather in Tokyo is sunny with a temperature of 22°C.


In [7]:
# Step 4: Continue the conversation with Anthropic
messages.append(ChatMessage(message=EasyInputMessageParam(role="user", content="What should I wear for that weather?")))

response3 = await router.create(
    input=messages,
    model="claude-haiku-4-5-20251001",
    max_output_tokens=16_000,
)
messages.extend(response3.output)

print("Anthropic response:")
print(extract_text(response3))

Anthropic response:
For 22°C and sunny weather in Tokyo, here are some clothing recommendations:

**Light layers are ideal:**
- **Top**: A light long-sleeve shirt, t-shirt, or short-sleeve top
- **Outerwear**: A light cardigan, denim jacket, or thin sweater (useful if it gets cooler in the evening)
- **Bottoms**: Lightweight pants, jeans, or shorts
- **Footwear**: Comfortable sneakers, casual shoes, or loafers

**Additional items to consider:**
- **Sunglasses and a hat** - protection from the sun
- **Sunscreen** - important since it's sunny
- **Light scarf** - optional, but nice for layering
- **Small bag or backpack** - to carry essentials

**General tips:**
- 22°C is mild and pleasant, so you don't need heavy winter clothing
- Since the sun is out, you'll likely feel warmer in direct sunlight
- It might be slightly cooler in the evening or in shaded areas, so layering is smart
- Comfortable walking shoes are great if you plan to explore the city

You'll be comfortable in casual, brea

## Parallel Function Calling

Models can call multiple tools in a single response, which requires special handling of reasoning depending on the provider which we handle transparently.


In [8]:
messages_parallel: list[ChatMessage] = [
    ChatMessage(
        message=EasyInputMessageParam(
            role="user",
            content="I need the weather in NYC (fahrenheit) and Apple's stock price. Call both tools.",
        )
    )
]

response_parallel = await router.create(
    input=messages_parallel,
    model="gpt-5.2",
    tools=TOOLS,
    tool_choice="required",
    instructions="You are a helpful assistant who calls tools.",
)

parallel_calls = extract_function_calls(response_parallel)
print(f"OpenAI made {len(parallel_calls)} parallel function calls:")
for call in parallel_calls:
    print(f"  {call['name']}({call['arguments']})")

OpenAI made 2 parallel function calls:
  get_weather({"location":"New York City, US","unit":"fahrenheit"})
  get_stock_price({"ticker":"AAPL"})


In [9]:
# Provide results for all parallel calls
messages_parallel.extend(response_parallel.output)

for call in parallel_calls:
    result = simulate_tool_execution(call["name"], call["arguments"])
    messages_parallel.append(
        ChatMessage(
            message=FunctionCallOutput(
                call_id=call["call_id"],
                output=result,
                type="function_call_output",
            )
        )
    )
    print(f"{call['name']} result: {result}")

get_weather result: {"temperature": 22, "unit": "fahrenheit", "conditions": "sunny", "location": "New York City, US"}
get_stock_price result: {"ticker": "AAPL", "price": 178.5, "currency": "USD"}


In [10]:
# Gemini interprets both results
response_parallel_2 = await router.create(input=messages_parallel, model="gemini-3-flash-preview")
messages_parallel.extend(response_parallel_2.output)

print("Gemini response:")
print(extract_text(response_parallel_2))

Gemini response:
The current weather in New York City is 22°F and sunny. Apple's stock price is $178.50.


In [11]:
# Continue with Anthropic
messages_parallel.append(
    ChatMessage(message=EasyInputMessageParam(role="user", content="Should I buy AAPL stock today given the weather?"))
)

response_parallel_3 = await router.create(
    input=messages_parallel,
    model="claude-haiku-4-5-20251001",
    max_output_tokens=16_000,
)
messages_parallel.extend(response_parallel_3.output)

print("Anthropic response:")
print(extract_text(response_parallel_3))

Anthropic response:
The weather and stock prices are independent of each other, so the sunny 22°F conditions in NYC shouldn't influence your investment decision for Apple stock.

When deciding whether to buy AAPL at $178.50, consider factors like:

- **Apple's fundamentals** - earnings, revenue growth, profit margins, competitive position
- **Market conditions** - overall market trends, interest rates, economic outlook
- **Your personal situation** - investment goals, time horizon, risk tolerance, portfolio diversification
- **Valuation** - is $178.50 a fair price relative to earnings and growth prospects?
- **Technical analysis** - if you use that approach, support/resistance levels, trends

I'd recommend researching Apple's latest earnings reports, analyst ratings, and your own investment strategy rather than weather conditions. If you're looking for stock advice, consider consulting a financial advisor who can assess your specific situation.


## Built-in Web Search

InteropRouter supports built-in web search via `WebSearchToolParam`. OpenAI, Gemini, and Anthropic all have native web search capabilities that InteropRouter maps to a unified interface.

Use `include=["web_search_call.results", "web_search_call.action.sources"]` to get search metadata in the response.


In [12]:
from openai.types.responses import WebSearchToolParam

# Continues the conversation from above
messages_parallel.append(
    ChatMessage(message=EasyInputMessageParam(role="user", content="Now can you look up one latest article about AI?"))
)

# Gemini performs web search
response_search = await router.create(
    input=messages_parallel,
    model="gemini-3-flash-preview",
    tools=[WebSearchToolParam(type="web_search")],
    include=["web_search_call.results", "web_search_call.action.sources"],
)
messages_parallel.extend(response_search.output)

print("Gemini web search response:")
print(extract_text(response_search))

Gemini web search response:
A very recent and significant article (dated December 22, 2025) discusses a breakthrough from researchers at **Duke University** who have developed a new AI framework that can find simple, readable rules within extremely complex systems.

### Key Highlights of the Article:
*   **The Problem:** Scientists often struggle to find mathematical equations for systems with thousands of interacting variables, such as weather patterns, climate change, or complex biological signals.
*   **The AI's Capability:** Inspired by "dynamicists" like Isaac Newton, this AI analyzes data from evolving systems and reduces massive complexity into compact, understandable equations. It can take a system with hundreds of variables and distill it down to its core governing rules.
*   **Applications:** The method is being applied across physics, engineering, and biology. Researchers believe it will help "see through the chaos" of data to uncover fundamental laws of nature that were pre

In [13]:
# Anthropic summarizes
messages_parallel.append(
    ChatMessage(message=EasyInputMessageParam(role="user", content="Can you summarize that article in one sentence?"))
)

response_search_2 = await router.create(
    input=messages_parallel,
    model="claude-haiku-4-5-20251001",
    max_output_tokens=16_000,
)
messages_parallel.extend(response_search_2.output)

print("Anthropic summary:")
print(extract_text(response_search_2))

Anthropic summary:
Duke University researchers have developed an AI framework that can distill complex systems with thousands of variables into simple, readable mathematical equations, enabling better scientific understanding across physics, engineering, and biology.


In [14]:
# Back to Gemini for follow-up
messages_parallel.append(
    ChatMessage(message=EasyInputMessageParam(role="user", content="Thank you! What other topics are trending in AI?"))
)

response_search_3 = await router.create(
    input=messages_parallel,
    model="gemini-3-flash-preview",
    tools=[WebSearchToolParam(type="web_search")],
    include=["web_search_call.results", "web_search_call.action.sources"],
)
messages_parallel.extend(response_search_3.output)

print("Gemini follow-up:")
print(extract_text(response_search_3))

Gemini follow-up:
Beyond the breakthrough in **interpretable AI** (like the Duke University framework you mentioned), several other major themes are dominating the AI landscape this December 2025:

### 1. The "Agentic" Shift
The biggest trend right now is the transition from AI as a "chatbot" to AI as an **agent**. 
*   **Autonomous Actions:** New systems like **OpenAI’s "Operator"** and **Anthropic’s Claude 4.5** are being used to navigate web browsers, schedule appointments, and execute complex workflows independently.
*   **Enterprise Integration:** Companies are launching "agentic layers" (like Salesforce’s Agentforce) that allow businesses to deploy autonomous agents to handle customer service, logistics, and marketing without human intervention.

### 2. "Vibe Coding" & MCP
In the developer world, the focus has shifted toward making AI infrastructure more standardized and accessible:
*   **Vibe Coding:** This is a trending term for programming using purely natural language. New to